# AWS Bedrock

In this quickstart you will learn how to use AWS Bedrock with all the power of tracking + eval with TruLens.

Note: this example assumes you've set up your AWS SSO with the CLI. Different authentication methods may change the initial client set up, but the rest should remain the same.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truera/trulens/blob/main/trulens_eval/examples/models/langchain_bedrock.ipynb)

### Import from TruLens, Langchain and Boto3

In [ ]:
# ! pip install trulens_eval==0.10.0 langchain boto3

In [ ]:
import boto3
import os

from langchain.llms.bedrock import Bedrock
from langchain import LLMChain

from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

### Set keys

For this example we'll need a Huggingface Key

In [ ]:
os.environ["HUGGINGFACE_API_KEY"] = "..."

### Create the Bedrock client and the Bedrock LLM

In [ ]:
client = boto3.client(service_name="bedrock", region_name="us-east-1")

bedrock_llm = Bedrock(
        model_id="amazon.titan-tg1-large",
        client=client
    )

### Set up standard langchain app with Bedrock LLM

In [ ]:
template = "You are a helpful assistant."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
example_human = HumanMessagePromptTemplate.from_template("Hi")
example_ai = AIMessagePromptTemplate.from_template("Argh me mateys")
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, example_human, example_ai, human_message_prompt]
)
chain = LLMChain(llm=bedrock_llm, prompt=chat_prompt, verbose=True)

print(chain.run("What's the capital of the USA?"))

### Initialize Feedback Function(s)

TODO: feedback function using bedrock

In [ ]:
from trulens_eval import TruChain, Feedback, Huggingface, Tru
tru = Tru()

In [ ]:
# Initialize Huggingface-based feedback function collection class:
hugs = Huggingface()

# Define a language match feedback function using HuggingFace.
f_lang_match = Feedback(hugs.language_match).on_input_output()
# By default this will check language match on the main app input and main app
# output.

### Instrument chain for logging with TruLens

In [ ]:
truchain = TruChain(chain,
    app_id='Chain1_ChatApplication',
    feedbacks=[f_lang_match])

In [ ]:
# Instrumented chain can operate like the original:
llm_response = truchain("What's the capital of the USA?")

display(llm_response)

### Explore in a Dashboard

In [ ]:
tru.run_dashboard() # open a local streamlit app to explore

# tru.stop_dashboard() # stop if needed

Alternatively, you can run `trulens-eval` from a command line in the same folder to start the dashboard.

### Or view results directly in your notebook

In [ ]:
tru.get_records_and_feedback(app_ids=[])[0] # pass an empty list of app_ids to get all